In [2]:
address_collection = Web3.toChecksumAddress("0x659a4bdaaacc62d2bd9cb18225d9c89b5b697a5a")
address_collection

'0x659A4BdaAaCc62d2bd9Cb18225D9C89b5B697A5A'

In [1]:
%run ConectAndAssignAccount.py

connected: True
address: 0x536A46C01c78df44e05A31eEb6aaFB85cddE20D5


In [3]:
# Alchamey requests api;
base_url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{apiKey}"  # apiKey loaded from environment in ConnectAndAssignAccount

# collection -> owners & owned tokens
def getOwnersForCollection():
    # docs: https://docs.alchemy.com/reference/getownersforcollection
    # first page
    first_page_url = base_url+ f"/getOwnersForCollection?contractAddress={address_collection}&withTokenBalances=true"
    headers = {"accept": "application/json"}
    response = requests.get(first_page_url, headers=headers) 

    # response.json()['ownerAddresses'] is a list with len() usually around 50k
    owner_addresses_page = response.json()['ownerAddresses']
    owner_addresses_all = owner_addresses_page

    # subsequent pages (todo:add more conditions avoid infinite loops)
    while True:
        page_key_old = response.json()['pageKey']
        next_page_url = first_page_url + f"&pageKey={page_key_old}"
        response = requests.get(next_page_url, headers=headers) 
        owner_addresses_page = response.json()['ownerAddresses']
        owner_addresses_all+=owner_addresses_page
        print(len(owner_addresses_all),page_key_old) # feedback to monitor
        if 'pageKey' not in response.json():
            print('no more page keys')
            return owner_addresses_all

# owner -> NFTs owned
# https://docs.alchemy.com/reference/getnfts
def getNFTs(owner):
    first_page_url = base_url + f"/getNFTs?owner={owner}&contractAddresses[]={address_collection}&withMetadata=true"
    headers = {"accept": "application/json"}
    response = requests.get(first_page_url, headers=headers)
    response_json = response.json()

    owner_nfts_all = response_json['ownedNfts']
    while len(owner_nfts_all) < response_json['totalCount']:
            page_key = response_json['pageKey']
            next_page_url = first_page_url + f"&pageKey={page_key}"
            response = requests.get(next_page_url, headers=headers) # introduces dupes bc of page size being 100, need to dedupe later
            response_json=response.json()
            owner_nfts_all += response_json['ownedNfts']
            print(f"{len(owner_nfts_all)} / {response_json['totalCount']}",page_key) # feedback to monitor

    return owner_nfts_all # may have dupes -- could dedupe by processing into a dict keyed by tokenId

def process_nft_list(nft_list):
    nfts_df = pd.DataFrame()
    for nft in nft_list:
        if 'metadata' in nft.keys():
            if 'attributes' in nft['metadata'].keys():
                if nft['metadata']['attributes']:
                    tmp_df=pd.DataFrame(nft['metadata']['attributes']).set_index('trait_type')
                    tmp_df=tmp_df.T.loc[['value']] # transpose for easier formatting
                else: tmp_df=pd.DataFrame()
            else: tmp_df=pd.DataFrame()
        else: tmp_df=pd.DataFrame()

        if 'title' in nft.keys(): 
            tmp_df['title'] = nft['title']
        else: tmp_df['title'] = None

        if 'id' in nft.keys():
            if 'tokenId' in nft['id'].keys():
                tmp_df['id'] = nft['id']['tokenId']
        else: tmp_df['id'] = None

        # if 'owner' in nft.keys():
        #     tmp_df['owner']=owner
        # else: tmp_df['owner']=owner

        nfts_df=nfts_df.append(tmp_df)

    nfts_df=nfts_df.drop_duplicates()
    return nfts_df

# process the list of nfts with metadata and dupes

# collection -> NFT info
# https://docs.alchemy.com/reference/getnftsforcollection
# skipped bc relevant info was returned by getNFTs
# # getNFTsForCollection
# url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{apiKey}/getNFTsForCollection?contractAddress={ens_nft_address}&withMetadata=true"
# headers = {"accept": "application/json"}
# response = requests.get(url, headers=headers)

# NFT sales
# https://docs.alchemy.com/reference/getnftsales
